15.1 A portfolio manager intends to launch a strategy that targets anannualized SR of  
2. Bets have a precision rate of 60%,with weekly frequency.The exit conditions  
are 2% for profit-taking, and –2% for stop-loss.  

(a) Is this strategy viable?

원하는 sharpe에 도달하지 못하였으므로 생존이 불가능하다.


In [19]:
def get_annualized_sharpe_ratio(precision, pt, sl, number_of_trades, **kwargs):
    expected_return = precision * pt + (1 - precision) * sl
    variance = precision * pt**2 + (1 - precision) * sl**2 - expected_return**2
    sharpe_ratio = expected_return / variance**0.5
    annualized_sharpe_ratio = sharpe_ratio * (number_of_trades**0.5)
    return annualized_sharpe_ratio

precision=0.6
pt=0.02
sl=-0.02

annualized_sharpe_ratio = get_annualized_sharpe_ratio(precision, pt, sl, 52)
print(f"Annualized Sharpe Ratio: {annualized_sharpe_ratio:.4f}")

Annualized Sharpe Ratio: 1.4720


(b) Ceteris paribus, what is the required precision rate that would make the
strategy profitable?

In [11]:
from sympy import *

def get_target_precision(target_sharpe_ratio, pt, sl, freq):
    a=(freq+target_sharpe_ratio**2)*(pt-sl)**2
    b=(2*freq*sl-target_sharpe_ratio**2*(pt-sl))*(pt-sl)
    c=freq*sl**2
    x=symbols('x')
    eq = Eq(a*x**2 + b*x + c, 0)
    solutions = solve(eq, x)
    solutions=[s for s in solutions if s.is_real and s > 0]
    if not solutions:
        return None
    return max(solutions)

target_precision=get_target_precision(2.0, 0.02, -0.02, 52)
print(f"Target Precision: {target_precision:.4f}")



Target Precision: 0.6336


(c) For what betting frequency is the target achievable?

In [13]:
def get_target_bet_size(target_sharpe_ratio, pt, sl, precision):
    expected_return = precision * pt + (1 - precision) * sl
    variance = precision * pt**2 + (1 - precision) * sl**2 - expected_return**2
    sharpe_ratio = expected_return / variance**0.5
    return round((target_sharpe_ratio/sharpe_ratio)**2)

target_bet_size = get_target_bet_size(2.0, 0.02, -0.02, 0.6)
print(f"Target Bet Size: {target_bet_size}")

Target Bet Size: 96


(d) For what profit-taking threshold is the target achievable?

In [15]:
def get_target_profit_taking(target_sharpe_ratio, precision, sl, freq):
    temp=target_sharpe_ratio*sqrt(precision*(1-precision))/sqrt(freq)-precision
    return sl*(1+1/temp)

target_profit_taking = get_target_profit_taking(2.0, 0.6, -0.02, 52)
print(f"Target Profit Taking: {target_profit_taking:.4f}")

Target Profit Taking: 0.0231


(e) What Would be an alternative stop-loss?

In [16]:
def get_target_stop_loss(target_sharpe_ratio, precision, pt, freq):
    temp=target_sharpe_ratio*sqrt(precision*(1-precision))/sqrt(freq)-precision
    return pt/(1+1/temp)

target_stop_loss = get_target_stop_loss(2.0, 0.6, 0.02, 52)
print(f"Target Stop Loss: {target_stop_loss:.4f}")

Target Stop Loss: -0.0173


15.2 Following up on the strategy from exercise 1.

(a) What is the sensitivity of SR to a 1% change in each parameter? 

precision에 가장 민감하다

In [22]:
import inspect

def get_partial_derivative(precision, pt, sl, number_of_trades, diff_ratio=0.01, target_parameter='pt'):
    frame=inspect.currentframe()
    args, _, _, values = inspect.getargvalues(frame)
    if target_parameter not in args:
        raise ValueError(f"Parameter '{target_parameter}' not found in function arguments.")
    
    original_target=get_annualized_sharpe_ratio(precision, pt, sl, number_of_trades)
    param_map={arg: values[arg] for arg in args}
    param_map[target_parameter] *= (1 + diff_ratio)
    new_target=get_annualized_sharpe_ratio(**param_map)
    param_map[target_parameter] /= (1 + diff_ratio)
    param_map[target_parameter] *= (1 - diff_ratio)
    new_target2=get_annualized_sharpe_ratio(**param_map)
    return ((original_target-new_target2)/diff_ratio+(new_target-new_target2)/diff_ratio)/2

for param in ['precision', 'pt', 'sl', 'number_of_trades']:
    diff = get_partial_derivative(0.6, 0.02, -0.02, 52, target_parameter=param)
    print(f"Difference for {param}: {diff:.4f}")

Difference for precision: 13.7836
Difference for pt: 5.5292
Difference for sl: -5.5292
Difference for number_of_trades: 1.1049


(b) Given these sensitivities,and assuming that all parameters are equally hard  
to improve, which one offers the lowest hanging fruit?  

모두 개선하기는 난이도가 비슷하다면 Precision을 개선하는 것이 가장 쉬운 방법일 것이다.

(c) Does changing any of the parameters in exercise 1 impact the others ? For
example, does changing the betting frequency modify the precision rate,
etc.?

그럴수 있을 것이다. precision을 개선하기 위해서 특정정보를 사용한다고 가정했을때 해당 정보의 주기가 현재 거래 주기보다 길때 freq를 더 길게 가져가게 될 수 있다.

15.3 Suppose a strategy that generates monthly bets over two years, with returns  
following a mixture of two Gaussian distributions. The first distribution has  
a mean of –0.1 and a standard deviation of 0.12. The second distribution has  
a mean of 0.06 and a standard deviation of 0.03. The probability that a draw  
comes from the first distribution is 0.15.

(a) Following L´ opez de Prado and Peijan[2004] and L´ opezde Prado and Fore-
man [2014], derive the first four moments for the mixture’s returns.

In [23]:
from ef3m import EF3M

mu1=-0.1
sigma1=0.12
mu2=0.06
sigma2=0.03
p1=0.15

ef3m_instance=EF3M()
moments=ef3m_instance.get_moments(mu1, sigma1, mu2, sigma2, p1)
moments_names=['mean', 'variance', 'skewness', 'kurtosis', '5th moment']
for name, value in zip(moments_names, moments):
    print(f"{name.capitalize()}: {value:.4f}")

Mean: 0.0870
Variance: 0.0150
Skewness: 0.0014
Kurtosis: 0.0003
5th moment: 0.0001


(b) What is the annualized SR?

In [24]:
sr=moments[0] / moments[1]**0.5
number_of_trades=24
asr=sr * (number_of_trades**0.5)
print(f"Annualized Sharpe Ratio: {asr:.4f}")

Annualized Sharpe Ratio: 3.4748


(c) Using those moments, compute PSR[1] (see Chapter 14). At a 95% confi-
dence level, would you discard this strategy?

PSR이 99.52%이므로 strategy를 discard하지 않을 것이다.

In [30]:
from scipy.stats import norm
import numpy as np

def get_psr(bm_sr, moments, number_of_trades):
    ob_mean, ob_variance, ob_skewness, ob_kurtosis, ob_5th_moment = moments
    ob_sr=ob_mean/ob_variance**0.5
    t_value=(ob_sr-bm_sr)*np.sqrt(number_of_trades-1)/np.sqrt(1-ob_skewness*ob_sr+(ob_kurtosis-1)/4*ob_sr**2)
    return norm.cdf(t_value)

psr=get_psr(1/np.sqrt(number_of_trades), moments, number_of_trades)
print(f"Probability of Superior Returns (PSR): {psr:.4f}")

Probability of Superior Returns (PSR): 0.9952


15.4 Using Snippet15.5, compute P[p < p𝜃∗=1] for the strategy described in exercise  
3. At a significance level of 0.05, would you discard this strategy ? Is this result  
consistent with PSR[𝜃∗]?  

In [ ]:
def get_mix_gaussian_samples(mu1, sigma1, mu2, sigma2, p1, n_samples):
        n1 = int(n_samples * p1)
        n2 = n_samples - n1
        samples1 = np.random.normal(mu1, sigma1, n1)
        samples2 = np.random.normal(mu2, sigma2, n2)
        ret=np.concatenate((samples1, samples2))
        np.random.shuffle(ret)
        return ret

def get_prob_failure(target_sr, freq, samples:np.ndarray):
    mean_pos, mean_neg = np.mean(samples[samples > 0]), np.mean(samples[samples <= 0])
    precision=samples[samples > 0].shape[0]/samples.shape[0]

    